In [5]:
import os
import os.path as op
import json
import cv2
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

In [9]:
root_dir = '/media/stopmosk/data/huawei'

img_root = op.join(root_dir, 'datasets_orig/huawei_1000/images')
gen_root = op.join(root_dir, 'output_dl2')

gen_filename = 'pred.huawei_1000.test.beam5.max20.odlabels.eval.json'  #_coco_format.json'

gen_fullpaths = []
gen_fullpaths.append(op.join(gen_root, 'agg_1cc_10coco_base_xe1', 'checkpoint-3-350000', gen_filename))
gen_fullpaths.append(op.join(gen_root, 'cc_base_xe_2', 'checkpoint-15-200000', gen_filename))
gen_fullpaths.append(op.join(gen_root, 'a100_agg_1', 'checkpoint-24-23000', gen_filename))
gen_fullpaths[-1]

'/media/stopmosk/data/huawei/output_dl2/a100_agg_1/checkpoint-24-23000/pred.huawei_1000.test.beam5.max20.odlabels.eval.json'

In [12]:
res_dicts = []

for gen_fullpath in gen_fullpaths[1:]:
    with open(gen_fullpath) as fp:
        res_json = json.load(fp)

    res_dict = {s['image_id']: s['caption'] for s in res_json}
    res_dicts.append(res_dict)
    #key = list(res_dict.keys())[0]
    #res_dict[key]
    # res_dict


FileNotFoundError: [Errno 2] No such file or directory: '/media/stopmosk/data/huawei/output_dl2/cc_base_xe_2/checkpoint-15-200000/pred.huawei_1000.test.beam5.max20.odlabels.eval.json'

In [59]:
def show_res(sample_idx, show=True, save=False):
    im_filename = sample_idx
    im_fullpath = os.path.join(img_root, im_filename)
    im = Image.open(im_fullpath)
    im.thumbnail((600, 400))

    plt.figure(figsize=(16,10))
    draw = ImageDraw.Draw(im)

    text_cap = f"Pred: {res_dict[sample_idx]}"
    #print('\n' + text_cap)

    max_str_len = im.width // 6  # max string length in letters
    strings_num = len(text_cap) // max_str_len + 1

    b_text = [0, 0, im.width, 12 * strings_num]
    draw.rectangle(b_text, fill=255)

    text_cap_chunks = [text_cap[seek: seek + max_str_len] for seek in range(0, len(text_cap), max_str_len)]
    for i, chunk in enumerate(text_cap_chunks):
        draw.text((0, i * 12), chunk, stroke_width=40, stroke_fill=255)

    plt.axis('off')
    
    if show:
        plt.imshow(im);
    
    if save:
        plt.savefig(op.join(gen_root, 'gen_files', sample_idx), bbox_inches='tight')
        plt.close()


In [60]:
%matplotlib inline

for sample_idx in tqdm(list(res_dict.keys())[0:]):
    show_res(sample_idx, save=True)

100%|██████████| 998/998 [02:19<00:00,  7.17it/s]
